In [6]:
import numpy as np
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch
import re

In [7]:
# Настройки подключения к PostgreSQL
DB_CONFIG = {
    "dbname": "bank_data",
    "user": "postgres",
    "password": "cjkysirj22",
    "host": "localhost",
    "port": "5432"
}

In [8]:
# Функция для преобразования типов (из NumPy → Python)
def convert_types(row):
    return tuple(
        int(x) if isinstance(x, (np.int64, np.int32)) else
        float(x) if isinstance(x, (np.float64, np.float32)) else
        bool(x) if isinstance(x, (np.bool_, bool)) else
        str(x) if isinstance(x, (np.datetime64, pd.Timestamp)) else
        x
        for x in row
    )

In [9]:
def insert_data(table_name, dataframe):
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()

    cols = ', '.join(dataframe.columns)
    values_template = ', '.join(['%s'] * len(dataframe.columns))
    query = f"INSERT INTO card_transactions.{table_name} ({cols}) VALUES ({values_template}) ON CONFLICT DO NOTHING;"

    # Конвертируем типы перед вставкой
    records = [convert_types(row) for row in dataframe.to_records(index=False)]

    execute_batch(cursor, query, records)

    conn.commit()
    cursor.close()
    conn.close()

# Загружаем датасет

In [ ]:
my_path = r"synthetic_fraud_data.csv"
df = pd.read_csv(my_path)
df.head()

,transaction_id,customer_id,card_number,timestamp,merchant_category,merchant_type,merchant,amount,currency,country,...,device,channel,device_fingerprint,ip_address,distance_from_home,high_risk_merchant,transaction_hour,weekend_transaction,velocity_last_hour,is_fraud
0,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820+00:00,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,...,iOS App,mobile,e8e6160445c935fd0001501e4cbac8bc,197.153.60.199,0,False,0,False,"{'num_transactions': 1197, 'total_amount': 334...",False
1,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464+00:00,Entertainment,gaming,Steam,3368.97,BRL,Brazil,...,Edge,web,a73043a57091e775af37f252b3a32af9,208.123.221.203,1,True,0,False,"{'num_transactions': 509, 'total_amount': 2011...",True
2,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762+00:00,Grocery,physical,Whole Foods,102582.38,JPY,Japan,...,Firefox,web,218864e94ceaa41577d216b149722261,10.194.159.204,0,False,0,False,"{'num_transactions': 332, 'total_amount': 3916...",False
3,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466+00:00,Gas,major,Exxon,630.60,AUD,Australia,...,iOS App,mobile,70423fa3a1e74d01203cf93b51b9631d,17.230.177.225,0,False,0,False,"{'num_transactions': 764, 'total_amount': 2201...",False
4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063+00:00,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,...,Chrome,web,9880776c7b6038f2af86bd4e18a1b1a4,136.241.219.151,1,False,0,False,"{'num_transactions': 218, 'total_amount': 4827...",True


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7483766 entries, 0 to 7483765
Data columns (total 24 columns):
 #   Column               Dtype  
---  ------               -----  
 0   transaction_id       object 
 1   customer_id          object 
 2   card_number          int64  
 3   timestamp            object 
 4   merchant_category    object 
 5   merchant_type        object 
 6   merchant             object 
 7   amount               float64
 8   currency             object 
 9   country              object 
 10  city                 object 
 11  city_size            object 
 12  card_type            object 
 13  card_present         bool   
 14  device               object 
 15  channel              object 
 16  device_fingerprint   object 
 17  ip_address           object 
 18  distance_from_home   int64  
 19  high_risk_merchant   bool   
 20  transaction_hour     int64  
 21  weekend_transaction  bool   
 22  velocity_last_hour   object 
 23  is_fraud             bool   
dty

In [12]:
df.describe()

,card_number,amount,distance_from_home,transaction_hour
count,7.483766e+06,7.483766e+06,7.483766e+06,7.483766e+06
mean,4.222100e+15,4.792468e+04,3.220519e-01,1.215467e+01
std,2.341170e+15,1.775562e+05,4.672628e-01,6.536767e+00
min,3.700086e+14,1.000000e-02,0.000000e+00,0.000000e+00
25%,4.004400e+15,3.635300e+02,0.000000e+00,7.000000e+00
50%,5.010745e+15,1.177450e+03,0.000000e+00,1.200000e+01
75%,5.999914e+15,2.242953e+04,1.000000e+00,1.800000e+01
max,6.999728e+15,6.253153e+06,1.000000e+00,2.300000e+01


### Подготовка и заливка таблицы customers

In [13]:
df['customer_id'] = df['customer_id'].apply(lambda x: re.sub(r'\D', '', str(x)))
df['customer_id']

0          72886
1          70474
2          10715
3          16193
4          87572
           ...  
7483761    11344
7483762    38961
7483763    75576
7483764    82508
7483765    20139
Name: customer_id, Length: 7483766, dtype: object

In [14]:
customers_df = df[['customer_id', 'card_number', 'card_type']].drop_duplicates()

In [ ]:
insert_data("customers", customers_df)

### Подготовка и заливка таблицы merchants

In [16]:
unique_merchants = df[['merchant', 'merchant_category', 'merchant_type', 'high_risk_merchant']].drop_duplicates().shape[0]
print(f"Уникальных сочетаний: {unique_merchants}")
df[['merchant', 'merchant_category', 'merchant_type', 'high_risk_merchant']].drop_duplicates().sort_values('merchant')


Уникальных сочетаний: 105


,merchant,merchant_category,merchant_type,high_risk_merchant
27,AMC Theaters,Entertainment,events,True
72,Airbnb,Travel,booking,True
62,Aldi,Grocery,physical,False
129,AliExpress,Retail,online,False
56,Amazon,Retail,online,False
...,...,...,...,...
7,Westin,Travel,hotels,True
2,Whole Foods,Grocery,physical,False
73,Xbox Live,Entertainment,gaming,True
9,eBay,Retail,online,False


In [ ]:
df[['merchant']].drop_duplicates().sort_values('merchant')

,merchant
27,AMC Theaters
72,Airbnb
62,Aldi
129,AliExpress
56,Amazon
...,...
7,Westin
2,Whole Foods
73,Xbox Live
9,eBay


In [18]:
# Создание merchant_id - превичного ключа
merchant_mapping = df[['merchant', 'merchant_category', 'merchant_type', 'high_risk_merchant']].drop_duplicates()
merchant_mapping.insert(0, 'merchant_id', range(1000, 1000 + len(merchant_mapping)))

# добавляем merchant_id в основной DataFrame
df = df.merge(merchant_mapping, on=['merchant'], how='left')
merchants_df = merchant_mapping.copy()


In [19]:
merchants_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 105 entries, 0 to 932
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   merchant_id         105 non-null    int64 
 1   merchant            105 non-null    object
 2   merchant_category   105 non-null    object
 3   merchant_type       105 non-null    object
 4   high_risk_merchant  105 non-null    bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 4.2+ KB


In [ ]:
insert_data("merchants", merchants_df)

### Заливка таблицы devices

In [21]:
# Создание device_id - превичного ключа
device_mapping = df[['device', 'device_fingerprint']].drop_duplicates()
device_mapping.insert(0, 'device_id', range(1000, 1000 + len(device_mapping)))

# добавляем device_id в основной DataFrame
df = df.merge(device_mapping, on=['device','device_fingerprint'], how='left')
devices_df = device_mapping.copy()


In [ ]:
insert_data("devices", devices_df)

### Заливка таблицы transactions

In [23]:
df["distance_from_home"] = df["distance_from_home"].astype(bool)

In [24]:
transactions_df = df[['transaction_id', 'customer_id', 'timestamp', 'transaction_hour', 'weekend_transaction',
                      'amount', 'currency', 'merchant_id', 'device_id', 'card_present',
                      'distance_from_home', 'ip_address']].drop_duplicates()

In [ ]:
insert_data("transactions", transactions_df)

### Подготовка и заливка таблицы velocity

In [26]:
for index, value in df['velocity_last_hour'].head(5).items():
    print(f"Строка {index}: {value}")

Строка 0: {'num_transactions': 1197, 'total_amount': 33498556.080464985, 'unique_merchants': 105, 'unique_countries': 12, 'max_single_amount': 1925480.6324148502}
Строка 1: {'num_transactions': 509, 'total_amount': 20114759.055250417, 'unique_merchants': 100, 'unique_countries': 12, 'max_single_amount': 5149117.011434267}
Строка 2: {'num_transactions': 332, 'total_amount': 39163854.72992601, 'unique_merchants': 97, 'unique_countries': 12, 'max_single_amount': 1852242.1831665323}
Строка 3: {'num_transactions': 764, 'total_amount': 22012599.81898404, 'unique_merchants': 105, 'unique_countries': 12, 'max_single_amount': 2055798.460682913}
Строка 4: {'num_transactions': 218, 'total_amount': 4827636.199648165, 'unique_merchants': 88, 'unique_countries': 12, 'max_single_amount': 1157231.252130005}


In [34]:
velocity_cols = ['num_transactions', 'total_amount', 'unique_merchants', 'unique_countries', 'max_single_amount']

# Раскрываем velocity_last_hour в отдельные колонки
df[velocity_cols] = df['velocity_last_hour'].apply(lambda x: eval(x) if isinstance(x, str) else {}).apply(pd.Series)
velocity_metrics_df = df[['transaction_id'] + velocity_cols].dropna()


In [35]:
velocity_metrics_df['total_amount'].describe()
velocity_metrics_df['max_single_amount'].describe()


count    7.483766e+06
mean     1.726432e+06
std      1.398325e+06
min      1.454232e-02
25%      8.015712e+05
50%      1.235738e+06
75%      2.290742e+06
max      6.253153e+06
Name: max_single_amount, dtype: float64

In [37]:
insert_data("velocity_metrics", velocity_metrics_df)

### Заливка таблицы if_fraud

In [36]:
is_fraud_df = df[['transaction_id', 'is_fraud']].drop_duplicates()
insert_data("fraud_status", is_fraud_df)